In [14]:
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Preparation;
open Microsoft.Quantum.Characterization;
open Microsoft.Quantum.Oracles;
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Convert;

In [15]:
// oracle f(x) = 1 if x = 00..00, 11..11
operation Oracle(groverQubits : Qubit[]) : Unit is Adj+Ctl {
    Controlled X(Most(groverQubits), Tail(groverQubits));
    within {
        ApplyToEachA(X, Most(groverQubits));
    }
    apply {
        Controlled X(Most(groverQubits), Tail(groverQubits));
    }
}

Oracle

In [16]:
// Grover diffusion operator
operation GroverDiffusion(groverQubits : Qubit[]) : Unit is Adj+Ctl {
    within {
        ApplyToEachA(H, Most(groverQubits));
        ApplyToEachA(X, Most(groverQubits));
    } apply {
        Controlled Z(Most(Most(groverQubits)), Tail(Most(groverQubits)));
    }
}

GroverDiffusion

In [17]:
operation GroverIteration(groverQubits : Qubit[]): Unit is Adj+Ctl {
    Oracle(groverQubits);
    GroverDiffusion(groverQubits);
}

GroverIteration

In [18]:
operation GroverPow(power: Int, groverQubits : Qubit[]): Unit is Adj+Ctl {
    for i in 1 .. power {
        GroverIteration(groverQubits);
    }
}

GroverPow

In [19]:
operation QuantumCounting(groverQubits : Qubit[], targetQubits : Qubit[]) : Unit is Adj+Ctl {
    let oracle = DiscreteOracle(GroverPow);
    QuantumPhaseEstimation(oracle, groverQubits, BigEndian(targetQubits));
}

QuantumCounting

In [24]:
operation MeasureTarget() : Int {
    use groverQubits = Qubit[6];
    use targetQubits = Qubit[8];
    
    QuantumCounting(groverQubits, targetQubits);
    
    let a = BoolArrayAsInt(Reversed(ResultArrayAsBoolArray(MultiM(targetQubits))));
    ResetAll(groverQubits);
    ResetAll(targetQubits);
    
    let theta = 2.0 * PI() * IntAsDouble(a) / IntAsDouble(2 ^ Length(targetQubits));
    let numSolutions= PowD(Cos(theta / 2.0), 2.0) * IntAsDouble(2 ^ (Length(groverQubits) - 1));
    Message($"a: {a}");
    return Round(numSolutions);
}

MeasureTarget

In [25]:
%simulate MeasureTarget

a: 0


32